In [1]:
#!/usr/bin/env python3
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd 
import astropy.units as u
from sunpy.time import TimeRange
import time
import sys

import datetime as dt
import matplotlib.dates as mdates
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LogNorm
from matplotlib import gridspec,transforms

%matplotlib inline

In [2]:
sys.path.insert(1, '../../20201013_jupiter/')
from sk import LofarRaw

In [29]:
filename = '../udpoutput/uranus-stokesVectors_0_2020-12-15T20:04:00_19629670898060' #0 is I and 1 is V I assume
frange = [15,30]
sbs = np.arange(76,198)
obs_mode = 3
time_len_mins = 176.
trange = TimeRange(filename.split('_')[-2], time_len_mins*u.min)
xlabel = "Time from {} {}".format(filename.split('_')[-2].split('T')[0], filename.split('_')[-2].split('T')[1])
ylabel = "Frequency (MHz)"
title = filename.split('/')[2]

In [ ]:
off_fname = '../udpoutput/offsource-stokesVectors_0_2020-12-15T20:04:00_19629670898060'
frange = [15,30]
sbs = np.arange(76,198)
obs_mode = 3
time_len_mins = 176.
offtrange = TimeRange(off_fname.split('_')[-2], time_len_mins*u.min)
#offxlabel = "Time from {} {}".format(off_fname.split('_')[-2].split('T')[0], off_fname.split('_')[-2].split('T')[1])
#offylabel = "Frequency (MHz)"
#offtitle = off_fname.split('/')[2]

In [30]:
raw = LofarRaw(fname=filename, sbs=sbs, obs_mode=obs_mode, frange=frange)
raw.data = raw.data[:,:78] ###the rest of the subbands are empty because of the way ilofar/realta records data into the data lanes

Time to read data: 0.000962s



In [31]:
flimits = np.array([0,78])
freqlimits = raw.sb_to_f(flimits+sbs[0], obs_mode)
xlims = list(map(dt.datetime.fromisoformat, [trange.start.value, trange.end.value]))
xlims = mdates.date2num(xlims)

In [32]:
datasplit = 10
normseconds = 42
norm_splits = int(((trange.minutes.value/datasplit)*60)/normseconds)

tchunks = trange.split(datasplit)
datas = np.array_split(raw.data, datasplit, axis=0)

In [33]:
for i,j in enumerate(datas):
    print("Analysing chunk #{} of data".format(i+1))

    arrs = np.array_split(j, norm_splits)
    print("\tSplitting sub-array")
    arrs = [(arr/np.quantile(np.abs(arr), 0.1, axis=0)) for arr in arrs] #normalise freq response
    #np.abs here so stokes V stays +/- and not all - or +
    print("\tSub-array normalised")
    d = np.vstack(arrs)

    #downsampling
    dec = 32
    summed = np.cumsum(d, axis=0)
    dec_data = summed[dec::dec, :] - summed[:-dec:dec, :]
    print('\tData downsampled x{}'.format(int(dec/2)))
    
    print("\tPlotting...\n")
    plot_data(dec_data, tchunks[i], freqlimits, xlabel, ylabel, 'stokesV_plots_freqnormed/Uranus_stokesI_plot_{}'.format(i+1))

Analysing chunk #1 of data
	Splitting sub-array
	Sub-array normalised
	Data downsampled x16
	Plotting...

Analysing chunk #2 of data
	Splitting sub-array
	Sub-array normalised
	Data downsampled x16
	Plotting...

Analysing chunk #3 of data
	Splitting sub-array
	Sub-array normalised
	Data downsampled x16
	Plotting...

Analysing chunk #4 of data
	Splitting sub-array
	Sub-array normalised
	Data downsampled x16
	Plotting...

Analysing chunk #5 of data
	Splitting sub-array
	Sub-array normalised
	Data downsampled x16
	Plotting...

Analysing chunk #6 of data
	Splitting sub-array
	Sub-array normalised
	Data downsampled x16
	Plotting...

Analysing chunk #7 of data
	Splitting sub-array
	Sub-array normalised
	Data downsampled x16
	Plotting...

Analysing chunk #8 of data
	Splitting sub-array
	Sub-array normalised
	Data downsampled x16
	Plotting...

Analysing chunk #9 of data
	Splitting sub-array
	Sub-array normalised
	Data downsampled x16
	Plotting...

Analysing chunk #10 of data
	Splitting sub-arr

In [56]:
raw = LofarRaw(fname=filename, sbs=sbs, obs_mode=obs_mode, frange=frange)

Time to read data: 0.001239s



In [57]:
data_fraction = 100 #the inverse of how much data to take at a time

In [58]:
raw.data = raw.data[::data_fraction]
raw.data = raw.data[:,:78]

In [59]:
raw.data.shape

(1289063, 78)

In [60]:
plt.figure(figsize=(12,6))
im = plt.imshow(raw.data.T, aspect='auto', origin='lower',
            vmin=np.nanpercentile(raw.data.T, 5), 
            vmax=np.nanpercentile(raw.data.T, 90),
            extent=[xlims[0], xlims[1], 
                    freqlimits[0].value, freqlimits[1].value])

plt.gca().xaxis_date()
date_format = mdates.DateFormatter('%H:%M:%S')
plt.gca().xaxis.set_major_formatter(date_format)

plt.title("Uranus observation - Stokes I")
plt.ylabel(ylabel, fontsize=14)
plt.xlabel(xlabel, fontsize=14)

divider = make_axes_locatable(plt.gca())
cax = divider.append_axes("right", size="5%", pad=0.05)
plt.colorbar(im, cax=cax)

plt.tight_layout()

In [61]:
plt.savefig('first_explore.png')

In [27]:
def plot_data(data, trange, flims, xlabel, ylabel, plotname):
    f = plt.figure(figsize=(12,6))
    f.set_facecolor('white')

    spec = gridspec.GridSpec(ncols=2, nrows=2, width_ratios=[6,1], height_ratios=[1,4])
    ax0 = f.add_subplot(spec[0])
    ax2 = f.add_subplot(spec[2])
    ax3 = f.add_subplot(spec[3])

    xs = data.T.sum(axis=0)
    ys = data.T.sum(axis=1)

    ax0.plot(xs, lw=0.5)
    ax0.set(xticklabels=[], xlim=[0,None])
    ax0.tick_params(bottom=False)

    rot = transforms.Affine2D().rotate_deg(270)
    ax3.plot(ys[::-1], lw=0.5, transform=rot+plt.gca().transData)
    ax3.set(yticklabels=[])
    ax3.tick_params(left=False)

    xlims = list(map(dt.datetime.fromisoformat, [trange.start.value, trange.end.value]))
    xlims = mdates.date2num(xlims)

    im = ax2.imshow(data.T, aspect='auto', origin='lower',
            vmin=np.nanpercentile(data.T, 5), 
            vmax=np.nanpercentile(data.T, 90),
            extent=[xlims[0], xlims[1], 
                    flims[0].value, flims[1].value])

    ax2.xaxis_date()
    date_format = mdates.DateFormatter('%H:%M:%S')
    ax2.xaxis.set_major_formatter(date_format)

    ax2.set_title("Uranus observation - Stokes I")
    ax2.set_ylabel(ylabel, fontsize=14)
    ax2.set_xlabel(xlabel, fontsize=14)

    divider = make_axes_locatable(ax2)
    cax = divider.append_axes("right", size="3%", pad=0.05)
    plt.colorbar(im, cax=cax)

    plt.tight_layout()
    plt.savefig(plotname, dpi=900)